In [6]:
from ast import keyword
from audioop import add
import os
import io
import sys
import re
import glob
#from typing import final
from datetime import datetime
import cv2

import dateutil
from dateutil import parser
from prettyprinter import pprint
from google.cloud import vision
import argparse
from matplotlib import pyplot as plt
from matplotlib import patches as pch
import textdistance

class GoogleApiEngine:
    __actual_image = None
    __image_content = None
    __api_image = None
    __plot = None

    image_type = None
    image_page = None
    image_info = None
    def __init__(self, image_file):
        """
        Class constructor
        """
        pass 
    
    def initialize_image_content(self, image_file=None):
        """
        This module reads the image file given as input 
        AND, converts it do an image representation of the Google Vision API

        Parameters
        ----------

            image_fileaz (path): The image file (and its full path). Defaults to None.

        Return
        ------

            success (boolean) : Return True if image is initialized, else return False
        """
        success = False

        self.__client = vision.ImageAnnotatorClient()
        self.image_info = {}
        self.image_info['faces'] = {}
        self.image_info['logos'] = {}
        self.image_info['labels'] = {}
        self.image_info['text'] = {}

        assert (os.path.isfile(image_file)), "The image file does not exist (or, the location is incorrect)"

        # Store the path of the image
        # We use this variable in other functions below
        self.__actual_image = image_file

        # Read the image file
        with io.open(self.__actual_image, 'rb') as image:
            self.__image_content = image.read()
        
        success = True

        # Convert it to an internal representation
        success = self.create_api_image_representation()
        
        return success

    def create_api_image_representation(self):
        """
        The module converts an image file into an internal representation of the 
        Google vision API

        Return
        ------

            success (boolean) : Return True if image file is converted into internal
                                representation of the Google Vision API.
        """
        success = False
        # Convert the image to a Google Vision API's internal represenation
        self.__api_image = vision.Image(content=self.__image_content)
        success = True        
        return success

    def get_facial_information(self, show_plot=None):
        """
        This module reads all the facial information given in an image as input
        If there are multiple faces, then the API tries to give details for every face
        # NOTE: boudning boxes of all features are stored returned as response

        Return
        ------

            success (boolean) : Return True if facial information is identified, else False

        """
        success = False
        # Run the image through the facial features detection algorithm
        # Note: This function can detect mulptiple faces
        self.__facial_response = self.__client.face_detection(image=self.__api_image)
        
        # Get the Face annotations into a list, for multiple faces 
        faces = self.__facial_response.face_annotations
        if len(faces) > 0:
            success = True

        # VARIABLES
        # Emotional probability labels
        possibility = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE', 'LIKELY', 'VERY_LIKELY')
        
        # Declare a plot 
        if self.__plot is None: 
            self.__plot = plt.imread(self.__actual_image)
        
        self.image_info['faces'] = {}

        if success: 
            for i, face in enumerate(faces):
                
                vertices = ([(vertex.x, vertex.y) for vertex in face.bounding_poly.vertices])
            
                # print('Vertices covering face: {}\n\n'.format(vertices))
                rect = pch.Rectangle(vertices[0], (vertices[1][0] - vertices[0][0]),
                                    (vertices[2][1] - vertices[0][1]), linewidth = 1,
                                    edgecolor='r', facecolor='none')

                # # Put the vertice-based rectangle on the image
                # if show_plot:
                #     ax.add_patch(rect)

                # Storing the data to image info variable
                face_dict = {}
                face_dict['anger'] = format(possibility[face.anger_likelihood])
                face_dict['joy'] = format(possibility[face.joy_likelihood])
                face_dict['surprise'] = format(possibility[face.surprise_likelihood])
                face_dict['sorrow'] = format(possibility[face.sorrow_likelihood])
                face_dict['detection_confidence'] = format(round(face.detection_confidence * 100, 2))
                face_dict['face_boundary'] = rect

                self.image_info['faces'][i] = face_dict

            if show_plot:
                plt.show()
            # plt.savefig(os.path.basename(self.__actual_image))

        return success

    def get_logo_information(self, show_plot=None):
        """
        Parameters
        ----------

            show_plot ([type], optional): [description]. Defaults to None.

        Returns
        -------

            success (boolean) : Return True if logo information is identified, else False.
        """
        success = False
        
        
        # Declare a plot 
        if self.__plot is None: 
            self.__plot = plt.imread(self.__actual_image)
        # Declaring an axis for the plot 
        # fig, ax = plt.subplots(1)
        # ax.imshow(self.__plot)

        self.__logo_response = self.__client.logo_detection(image=self.__api_image)
        logos = self.__logo_response.logo_annotations
        if len(logos) > 0:
            success = True

        self.image_info['logos'] = {}
        if success:
            for i, logo in enumerate(logos):

                vertices = ([(vertex.x, vertex.y) for vertex in logo.bounding_poly.vertices])

                # print('Vertices covering Logo: {}\n\n'.format(vertices))
                rect = pch.Rectangle(vertices[0], (vertices[1][0] - vertices[0][0]),
                                    (vertices[2][1] - vertices[0][1]), linewidth = 1,
                                    edgecolor='b', facecolor="none")

                # # Put the vertice-based rectangle on the image
                # if show_plot:
                #     ax.add_patch(rect)

                # Storing the data to image info variable
                logo_dict = {}
                logo_dict['logo_boundary'] = rect
                logo_dict['detection_confidence'] = round(logo.score * 100 ,2)
                logo_dict['logo_authority'] = logo.description

                self.image_info['logos'][i] = logo_dict
        

            if show_plot:
                plt.show()
            # plt.savefig(os.path.basename(self.__actual_image))

        return success

    def get_label_description(self):
        """
        Module to return the label description of the image

        Return
        -------

            success (boolean) : Return True if label description is identified, else False
        """
        success = False

        self.__label_response = self.__client.label_detection(image=self.__api_image)
        labels = self.__label_response.label_annotations
        if len(labels) > 0:
            success = True

        self.image_info['document_description'] = {}
        if success: 
            for i, label in enumerate(labels):
                if label.description == "Font":
                    self.image_info['document_description']['has_text'] = label.score
                    self.image_info['document_description']['has_text'] = True
                if label.description == "Rectangle":
                    self.image_info['document_description']['has_shape'] = label.description
                    self.image_info['document_description']['shape_confidence'] = round(label.score, 2)
                if label.description == "Signage":
                    self.image_info['document_description']['has_signature'] = label.score
                    self.image_info['document_description']['has_signature'] = True
                if label.description == "Paper product":
                    self.image_info['document_description']['document_build'] = label.description

        return success

    def get_text_description(self, show_plot=None):
        """
        Module to extract text from the image
        """
        success = False
        # if self.__plot is None: 
        #     self.__plot = plt.imread(self.__actual_image)

        self.__text_response = self.__client.document_text_detection(image=self.__api_image, 
                                                            image_context={"language_hints": ["en", "ar"]})
        self.doctexts = self.__text_response.full_text_annotation.pages
        return True

    def run(self, image_file):
        
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = os.path.join("C:/air_ticket/emirati_id", 'creds.json')
        # Intitalize the vision annotator
        self.__client = vision.ImageAnnotatorClient()
        # The image information from various sub-process. 
        # This variable can be stored as a JSON variable in the end
        self.image_info = {}
        self.image_info['faces'] = {}
        self.image_info['logos'] = {}
        self.image_info['labels'] = {}
        self.image_info['text'] = {}
        success = self.initialize_image_content(image_file=self.image_file)
        if not success:
            print("Could not read the image")
        else:
            facial_success = self.get_facial_information()
            logo_success = self.get_logo_information()
            label_success = self.get_label_description()
            text_success = self.get_text_description()
            return self.doctexts


In [92]:
%%writefile updated_emirati_id.py

import json

class EmiratesIDCard(GoogleApiEngine):
    

    def __init__(self: object, image_file):
        '''
        Constructs all the necessary attributes for emirates ID card object
       
        Parameters
        -----------
            img_obj (ImageProcessing) :: ImageProcessing object to access its attributes 

        Attributes
        -----------
            image_info (dict) :: Stores image_info attribute of ImageProcessing object to extend it with Emirates ID Card Attributes 
            
            doctexts (text object) :: Stores text of all pages
        '''
        self.image_file = image_file

    def get_name(self: object) -> str:

        self.blocks = []
        self.words = []
        # check if it is the front page
        # TODO if face annotation is true, the side of the aadhaar card is front

        # looping through each page in the document
        for i, page in enumerate(self.doctexts[:]):

            # looping through each block on the page
            for j, block in enumerate(page.blocks[:]):

                # looping through each paragraph in the block
                for k, paragraph in enumerate(block.paragraphs):

                    # looping through each word in the paragraph
                    for a, word in enumerate(paragraph.words):

                        # concatenating symbols to store the word in variable word_text
                        word_text = ''.join([
                                    symbol.text for symbol in word.symbols
                                    ])

                        self.blocks.append((j, word_text, word.confidence))
                        self.words.append(word_text)
                        # print(j, word_text, word.confidence)
        
        print(self.words)

        try:
            name = ""
            flag = 0

            for i in range(len(self.blocks)):
                if self.blocks[i][1].lower() == "name":
                    flag = 1
                    # name is present after the key 'Name' and continues upto the word 'nationality'
                    for j in range(i+1, len(self.blocks)):
                        if self.blocks[j][1].lower() == "nationality":
                            break
                        # since the card has some arabic text, encode() is used to encode the string to UTF-8
                        elif self.blocks[j][1].encode().isalpha():
                            name += self.blocks[j][1] + " "  
                if flag == 1:
                    break

            self.image_info['text']['name'] = name.strip
            self.name = name.strip()
            return name.strip()

        except Exception as e:
            print(e)
            self.name = None
            return ("Name not found")

    
    def get_card_number(self: object) -> str:

        self.blocks = []
        # check if it is the front page
        # TODO if face annotation is true, the side of the aadhaar card is front

        # looping through each page in the document
        for i, page in enumerate(self.doctexts[:]):

            # looping through each block on the page
            for j, block in enumerate(page.blocks[:]):

                # looping through each paragraph in the block
                for k, paragraph in enumerate(block.paragraphs):

                    # looping through each word in the paragraph
                    for a, word in enumerate(paragraph.words):

                        # concatenating symbols to store the word in variable word_text
                        word_text = ''.join([
                                    symbol.text for symbol in word.symbols
                                    ])

                        self.blocks.append((j, word_text, word.confidence))

        try:
            id_number = ""
            flag = 0

            for i in range(len(self.blocks)):
                # id number is present after the key "idnumber"
                if (self.blocks[i][1]+self.blocks[i+1][1]).lower() == "idnumber":
                    flag = 1
                    # if string obtained after 'idnumber' is numeric, it is the id number
                    for j in range(i+1, len(self.blocks)):
                        if self.blocks[j][1][0].isnumeric():
                            id_number = self.blocks[j][1]
                # break the outer loop once the id number is fetched
                if flag == 1:
                    break

            self.image_info['text']['card_number'] = id_number
            self.card_number = id_number
            return id_number

        except Exception as e:
            print(e)
            self.id_number = None
            return ("ID number not found")

    def get_nationality(self: object) -> str:

        self.blocks = []
        # check if it is the front page
        # TODO if face annotation is true, the side of the aadhaar card is front

        # looping through each page in the document
        for i, page in enumerate(self.doctexts[:]):

            # looping through each block on the page
            for j, block in enumerate(page.blocks[:]):

                # looping through each paragraph in the block
                for k, paragraph in enumerate(block.paragraphs):

                    # looping through each word in the paragraph
                    for a, word in enumerate(paragraph.words):

                        # concatenating symbols to store the word in variable word_text
                        word_text = ''.join([
                                    symbol.text for symbol in word.symbols
                                    ])

                        self.blocks.append((j, word_text, word.confidence))

        try:
            nationality = ""
            flag = 0

            for i in range(len(self.blocks)):
                # nationality is present after the key 'nationality'
                if self.blocks[i][1].lower() == "nationality":
                    flag = 1

                    for j in range(i+2, len(self.blocks)):
                        # since the card has some arabic text,
                        # encode() is used to encode the string to UTF-8

                        # break the loop after any non alphabetic string is fetched
                        # else before it, the strings are a part of nationality
                        if not self.blocks[j][1].encode().isalpha():
                            break
                        nationality += self.blocks[j][1] + " "

                if flag == 1:
                    break

            self.image_info['text']['nationality'] = nationality.strip()
            self.nationality = nationality.strip()
            return nationality.strip()

        except Exception as e:
            print(e)
            self.nationality = None
            return ("Nationality not found")

    def is_arabic(self, word):
        regex = r"[\u0600-\u06FF]+"
        if re.findall(regex, word):
            return True
        return False

    def get_ar_name(self: object) -> str:

        self.blocks = []
        # check if it is the front page
        # TODO if face annotation is true, the side of the aadhaar card is front

        # looping through each page in the document
        for i, page in enumerate(self.doctexts[:]):

            # looping through each block on the page
            for j, block in enumerate(page.blocks[:]):

                # looping through each paragraph in the block
                for k, paragraph in enumerate(block.paragraphs):

                    # looping through each word in the paragraph
                    for a, word in enumerate(paragraph.words):

                        # concatenating symbols to store the word in variable word_text
                        word_text = ''.join([
                                    symbol.text for symbol in word.symbols
                                    ])

                        self.blocks.append((j, word_text, word.confidence))

        try:
            self.ar_name = ""
            flag = 0

            for i in range(len(self.blocks)):
                if self.blocks[i][1][0].isnumeric():
                    for j in range(i+3, len(self.blocks)):
                        if self.is_arabic(self.blocks[j][1]):
                            self.ar_name += self.blocks[j][1]+" "
                        if 'name' in self.blocks[j][1].lower():
                            flag = 1
                            break
                if flag:
                    break

            self.image_info['text']['ar_name'] = self.ar_name.strip()
            self.ar_name = self.ar_name.strip()
            return self.ar_name.strip()

        except Exception as e:
            print(e)
            self.ar_name = None
            return ("Name not found")

    def get_ar_nationality(self):

        self.blocks = []
        # check if it is the front page
        # TODO if face annotation is true, the side of the aadhaar card is front

        # looping through each page in the document
        for i, page in enumerate(self.doctexts[:]):

            # looping through each block on the page
            for j, block in enumerate(page.blocks[:]):

                # looping through each paragraph in the block
                for k, paragraph in enumerate(block.paragraphs):

                    # looping through each word in the paragraph
                    for a, word in enumerate(paragraph.words):

                        # concatenating symbols to store the word in variable word_text
                        word_text = ''.join([
                                    symbol.text for symbol in word.symbols
                                    ])

                        self.blocks.append((j, word_text, word.confidence))

        try:
            name = self.get_name()
            name = name.split()[-1]
            self.ar_nationality = ""
            flag = 0

            for i in range(len(self.blocks)):
                if name in self.blocks[i][1]:
                    for j in range(i+3, len(self.blocks)):
                        if 'nationality' in self.blocks[j][1].lower():
                            flag = 1
                            break
                        self.ar_nationality += self.blocks[j][1]
                if flag:
                    break

            self.image_info['text']['ar_nationality'] = self.ar_nationality.strip()
            self.ar_nationality = self.ar_nationality.strip()
            return self.ar_nationality.strip()

        except Exception as e:
            print(e)
            self.ar_nationality = None
            return ("Nationality not found")

    def get_all(self):
        self.doctexts = self.run(self.image_file)
        fp = open('keys.json', encoding='utf-8')
        keys = json.load(fp)
        self.all = {
            "English": {
                "Name": self.get_name(),
                "card_Number": self.get_card_number(),
                "Nationality": self.get_nationality()
            },
            "Arabic":{
                keys["Name"][0]: self.get_ar_name(),
                keys["ID Number"][0]: self.get_card_number(),
                keys["Nationality"][0]: self.get_ar_nationality()
            }
        }
        return self.all

Writing updated_emirati_id.py


In [91]:
image_file = 'C:/air_ticket/emirati_id/Emirati_ID_1.jpg'
obj = EmiratesIDCard(image_file)
print(obj.get_all())

['دولة', 'الإمارات', 'العربية', 'المتحدة', 'بطاقة', 'هوية', 'مقيم', 'United', 'Arab', 'Emirates', 'Resident', 'Identity', 'Card', 'رقم', 'الهوية', '/', 'ID', 'Number', '784-1978-7919035-9', 'الإسم', ':', 'محمد', 'زين', 'العابدين', 'محمد', 'الياس', 'Name', ':', 'Mohammed', 'Joynal', 'Abdin', 'Mohammed', 'Ilias', 'الجنسية', ':', 'بنغلاديش', 'Nationality', ':', 'Bangladesh']
['دولة', 'الإمارات', 'العربية', 'المتحدة', 'بطاقة', 'هوية', 'مقيم', 'United', 'Arab', 'Emirates', 'Resident', 'Identity', 'Card', 'رقم', 'الهوية', '/', 'ID', 'Number', '784-1978-7919035-9', 'الإسم', ':', 'محمد', 'زين', 'العابدين', 'محمد', 'الياس', 'Name', ':', 'Mohammed', 'Joynal', 'Abdin', 'Mohammed', 'Ilias', 'الجنسية', ':', 'بنغلاديش', 'Nationality', ':', 'Bangladesh']
{'English': {'Name': 'Mohammed Joynal Abdin Mohammed Ilias', 'card_Number': '784-1978-7919035-9', 'Nationality': 'Bangladesh'}, 'Arabic': {'اسم': 'محمد زين العابدين محمد الياس', 'رقم الهوية': '784-1978-7919035-9', 'جنسية': 'بنغلاديش'}}
